In [1]:
# Use Google Drive as Data Storage
from google.colab import drive
drive.mount('/content/gdrive')

!ls -l ./gdrive/MyDrive/Colab\ Notebooks/datasets

Mounted at /content/gdrive
total 164245
-rw------- 1 root root     30623 Dec  6 02:29 sample-submission.csv
-rw------- 1 root root  33075636 Dec  6 02:29 test_data.csv
-rw------- 1 root root 135047556 Dec  6 02:29 train_data.csv
-rw------- 1 root root     32350 Dec  6 02:29 train_target.csv


In [84]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import TensorDataset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [85]:
import pandas as pd
train_data_raw = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/train_data.csv', header=None).to_numpy()


train_target_raw = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/train_target.csv', header=None).to_numpy().flatten()

train_data = torch.from_numpy(train_data_raw).type(torch.float32).view(-1, 1, 48, 48)
train_target = torch.from_numpy(train_target_raw).type(dtype=torch.long)

test_data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/test_data.csv', header=None)

train_loader = DataLoader(TensorDataset(train_data,train_target), batch_size = 32, shuffle = True)



In [86]:
import torch
import torchvision
import torchvision.transforms as transforms

In [87]:
import torch.nn as nn
import torch.optim as optim
class FacialClassifier(nn.Module):
    def __init__(self):
        super(FacialClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 6 * 6, 128)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return x

In [89]:
# Initialize the model, loss function, and optimizer
model = FacialClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

FacialClassifier(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=4608, out_features=128, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=128, out_feature

In [90]:
from torch.optim import lr_scheduler

#Define a learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        imgs, targets = batch
        imgs = imgs.view(imgs.shape[0], 1, 48, 48)  # Reshape to (batch_size, channels, height, width)
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


    scheduler.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/8], Loss: 1.0520
Epoch [2/8], Loss: 0.9725
Epoch [3/8], Loss: 0.4506
Epoch [4/8], Loss: 0.4478
Epoch [5/8], Loss: 0.3086
Epoch [6/8], Loss: 0.4544
Epoch [7/8], Loss: 0.3169
Epoch [8/8], Loss: 0.4083


In [91]:
torch.save(model.state_dict(), "facial_classifier.pth")
test_data = torch.from_numpy(test_data.to_numpy()).type(torch.float32).view(-1, 1, 48, 48)

In [92]:
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in DataLoader(test_data, batch_size=32, shuffle=False):
        imgs = batch.to(device)
        predictions = model(imgs)
        _, predicted_labels = torch.max(predictions, 1)
        all_predictions.extend(predicted_labels.cpu().numpy())

# Save predictions to a CSV file in the specified format
submission_df = pd.DataFrame({'Id': range(len(all_predictions)), 'Category': all_predictions})
submission_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/datasets/predictions6.csv', index=False)